In [3]:
!pip install PyPDF2

ERROR: Could not find a version that satisfies the requirement PyPDF2 (from versions: none)
ERROR: No matching distribution found for PyPDF2


# **Step 1: Extract Content with PyPDF2**

In [5]:
import PyPDF2
import io
import requests

# Step 1: Download the PDF
pdf_url = 'https://drive.google.com/uc?export=download&id=16ROOOSBt6cUClyMZsKGtVhRlTMN1jKv7'
response = requests.get(pdf_url)

if response.status_code == 200:
    # Load the PDF into memory
    with io.BytesIO(response.content) as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        
        # Step 2: Extract text and metadata
        extracted_text = ""
        for page_num in range(len(reader.pages)):
            page = reader.pages[page_num]
            extracted_text += page.extract_text()  # Extracts text content

        # Extract basic metadata
        metadata = reader.metadata
        print("Metadata:", metadata)
        print("Extracted Text:", extracted_text)
else:
    print(f"Failed to download PDF. Status code: {response.status_code}")


Metadata: {'/Title': 'Agy Launcher for Young People - Google Forms', '/Creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36', '/Producer': 'Skia/PDF m128', '/CreationDate': "D:20240913104833+00'00'", '/ModDate': "D:20240913104833+00'00'"}
Extracted Text: Male
Female
Privet job
10 - 14 years
15 - 19 years
20 - 24 years
25 - 30 years
Dhaka
Chattogram
Khulna
Rajshahi
Barishal
Sylhet
Rangpur
Mymensingh
Kno?
Daily
Several times a week
Once a week
A few times a month
Rarely (once a month or less)
Never
50-60 years
60-70 years
70-80 years
80+ years
All about  Accessibility F eatur es and other Smar t Phone F eatur es:
These questions will help you share which accessibility features and other functions you ﬁnd most helpful when assisting older people with their mobile phones. Your insights will help us understand what works best and how to improve these features to support 
users better.
Some of the Smart Phone Usable Featur

In [9]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 40.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 90.8 MB/s eta 0:00:00:00:0100:01


# **Used pdfid.py to Analyze a PDF**

In [ ]:
import requests
import re

def download_pdf(pdf_url, pdf_path):
    """
    Download a PDF from a URL and save it locally.
    """
    response = requests.get(pdf_url)
    with open(pdf_path, 'wb') as f:
        f.write(response.content)
    print(f"PDF downloaded and saved as {pdf_path}")

def extract_js_from_pdf(pdf_path):
    """
    Extract JavaScript from the PDF by parsing the content.
    """
    js_codes = []

    # Read the PDF file
    with open(pdf_path, 'rb') as f:
        pdf_data = f.read().decode('latin1')  # Decoding for simplicity; adjust if needed

    # Regular expression patterns to identify JavaScript
    js_pattern = re.compile(r'/JavaScript\s*\[(.*?)\]', re.DOTALL)

    # Find all JavaScript objects
    matches = js_pattern.findall(pdf_data)

    for match in matches:
        js_code = re.sub(r'[\x00-\x1F]', '', match)  # Remove control characters
        js_codes.append(js_code.strip())

    return js_codes

def analyze_pdf(pdf_path):
    """
    Analyze the PDF to find potential malicious elements and extract JavaScript.
    """
    # Initialize counters
    counters = {
        'obj': 0,
        'endobj': 0,
        'stream': 0,
        'endstream': 0,
        'xref': 0,
        'trailer': 0,
        'startxref': 0,
        '/Page': 0,
        '/Encrypt': 0,
        '/ObjStm': 0,
        '/JS': 0,
        '/JavaScript': 0,
        '/AA': 0,
        '/OpenAction': 0,
        '/AcroForm': 0,
        '/JBIG2Decode': 0,
        '/RichMedia': 0,
        '/Launch': 0,
        '/EmbeddedFile': 0,
        '/XFA': 0,
        '/Colors > 2^24': 0
    }

    # Read the PDF file
    with open(pdf_path, 'rb') as f:
        pdf_data = f.read().decode('latin1')  # Decoding for simplicity; adjust if needed

    # Count occurrences of each keyword
    for key in counters:
        counters[key] = pdf_data.count(key)

    # Extract JavaScript code
    js_codes = extract_js_from_pdf(pdf_path)

    # Print the results
    for key, count in counters.items():
        print(f"{key:20} {count}")

    print("\nJavaScript found in PDF:")
    if js_codes:
        for js_code in js_codes:
            print(js_code)
    else:
        print("No JavaScript code found.")

if __name__ == '__main__':
    # URL of the PDF to analyze
    pdf_url = "https://drive.google.com/uc?export=download&id=12N-22vIqSR1lKe2RT9OmF7FCuDhI6TTw"  # Replace with your PDF link
    pdf_path = 'downloaded_pdf.pdf'

    # Download the PDF
    download_pdf(pdf_url, pdf_path)

    # Analyze the PDF
    analyze_pdf(pdf_path)


In [6]:
!pip install pdfminer.six

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 41.1 MB/s eta 0:00:0000:0100:01


In [12]:
import requests
import re
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdftypes import resolve1, stream_value
from pdfminer.psparser import PSLiteral

def download_pdf(pdf_url, pdf_path):
    """
    Download a PDF from a URL and save it locally.
    """
    response = requests.get(pdf_url)
    with open(pdf_path, 'wb') as f:
        f.write(response.content)
    print(f"PDF downloaded and saved as {pdf_path}")

def extract_js_from_obj(obj):
    """
    Extract JavaScript code from a PDF object if it contains /JavaScript or /JS.
    """
    js_code = []
    if isinstance(obj, dict):
        for key, value in obj.items():
            if isinstance(key, PSLiteral) and key.name in ('JS', 'JavaScript'):
                resolved_value = resolve1(value)
                if isinstance(resolved_value, bytes):
                    js_code.append(resolved_value.decode('latin1', errors='ignore'))
                elif isinstance(resolved_value, str):
                    js_code.append(resolved_value)
    return js_code

def extract_elements_from_pdf(pdf_path):
    """
    Extract various elements from the PDF that might contain malicious content.
    """
    elements = {
        'JavaScript': [],
        'EmbeddedFile': [],
        'Launch': [],
        'AdditionalActions': [],
        'Encrypt': False,
        'ObjStm': []
    }

    with open(pdf_path, 'rb') as f:
        parser = PDFParser(f)
        doc = PDFDocument(parser)

        # Check if the PDF is encrypted
        if doc.is_encrypted:
            try:
                doc.decrypt('')
            except Exception as e:
                print(f"Failed to decrypt PDF: {e}")
                elements['Encrypt'] = True
                return elements

        # Iterate through each page and extract its contents
        for page in PDFPage.create_pages(doc):
            # Resolve page annotations (like JS or additional actions)
            if page.annots:
                for annot in resolve1(page.annots):
                    if isinstance(annot, list):
                        for item in annot:
                            obj = resolve1(item)
                            js_code = extract_js_from_obj(obj)
                            if js_code:
                                elements['JavaScript'].extend(js_code)

            # Resolve page resources (like embedded files or JS in resources)
            if page.resources:
                res = resolve1(page.resources)
                for obj_key, obj_val in res.items():
                    js_code = extract_js_from_obj(obj_val)
                    if js_code:
                        elements['JavaScript'].extend(js_code)

    return elements

def analyze_pdf(pdf_path):
    """
    Analyze the PDF to find potential malicious elements and extract information.
    """
    # Extract elements
    elements = extract_elements_from_pdf(pdf_path)

    # Print the results
    print("\nJavaScript found in PDF:")
    if elements['JavaScript']:
        for js_code in elements['JavaScript']:
            print(js_code)
    else:
        print("No JavaScript code found.")

    print("\nEmbedded Files found in PDF:")
    if elements['EmbeddedFile']:
        for file in elements['EmbeddedFile']:
            print(file)
    else:
        print("No embedded files found.")

    print("\nLaunch Actions found in PDF:")
    if elements['Launch']:
        for launch in elements['Launch']:
            print(launch)
    else:
        print("No launch actions found.")

    print("\nAdditional Actions found in PDF:")
    if elements['AdditionalActions']:
        for action in elements['AdditionalActions']:
            print(action)
    else:
        print("No additional actions found.")

    print("\nEncryption status in PDF:")
    print("Encrypted" if elements['Encrypt'] else "Not encrypted")

if __name__ == '__main__':
    # URL of the PDF to analyze
    pdf_url = "https://drive.google.com/uc?export=download&id=12N-22vIqSR1lKe2RT9OmF7FCuDhI6TTw"  # Replace with your PDF link
    pdf_path = 'downloaded_pdf.pdf'

    # Download the PDF
    download_pdf(pdf_url, pdf_path)

    # Analyze the PDF
    analyze_pdf(pdf_path)


PDF downloaded and saved as downloaded_pdf.pdf


AttributeError: 'PDFDocument' object has no attribute 'is_encrypted'

In [15]:
pip install pdfid

Note: you may need to restart the kernel to use updated packages.


In [17]:
import requests
import subprocess
import os

def download_pdf(pdf_url, pdf_path):
    """
    Download a PDF from a URL and save it locally.
    """
    response = requests.get(pdf_url)
    with open(pdf_path, 'wb') as f:
        f.write(response.content)
    print(f"PDF downloaded and saved as {pdf_path}")

def analyze_pdf_with_pdfid(pdf_path):
    """
    Use pdfid.py to analyze the PDF for malicious elements like JavaScript, embedded files, and actions.
    """
    # Check if pdfid.py exists, if not download it
    pdfid_script = "pdfid.py"
    if not os.path.exists(pdfid_script):
        print(f"{pdfid_script} not found, downloading it...")
        pdfid_url = "https://raw.githubusercontent.com/DidierStevens/DidierStevensSuite/master/pdfid.py"
        response = requests.get(pdfid_url)
        with open(pdfid_script, 'w') as f:
            f.write(response.text)
    
    # Run pdfid.py on the PDF file
    try:
        result = subprocess.run(['python', pdfid_script, pdf_path], stdout=subprocess.PIPE)
        output = result.stdout.decode('utf-8')
        print("PDF Analysis Results:")
        print(output)
    except Exception as e:
        print(f"Error running pdfid.py: {e}")

if __name__ == '__main__':
    # URL of the PDF to analyze
    pdf_url = "https://drive.google.com/uc?export=download&id=12N-22vIqSR1lKe2RT9OmF7FCuDhI6TTw"  # Replace with your PDF link
    pdf_path = 'downloaded_pdf.pdf'

    # Download the PDF
    download_pdf(pdf_url, pdf_path)

    # Analyze the PDF using pdfid.py
    analyze_pdf_with_pdfid(pdf_path)

PDF downloaded and saved as downloaded_pdf.pdf
pdfid.py not found, downloading it...
PDF Analysis Results:
PDFiD 0.2.8 downloaded_pdf.pdf
 PDF Header: %PDF-1.4
 obj                  509
 endobj               509
 stream                15
 endstream             15
 xref                   1
 trailer                1
 startxref              1
 /Page                  2
 /Encrypt               0
 /ObjStm                0
 /JS                    1
 /JavaScript            1
 /AA                    0
 /OpenAction            1
 /AcroForm              0
 /JBIG2Decode           0
 /RichMedia             0
 /Launch                0
 /EmbeddedFile          0
 /XFA                   0
 /Colors > 2^24         0




In [19]:
!apt-get install -y poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpoppler118 poppler-data
Suggested packages:
  ghostscript fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanese-gothic | fonts-ipafont-gothic fonts-arphic-ukai
  fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  libpoppler118 poppler-data poppler-utils
0 upgraded, 3 newly installed, 0 to remove and 39 not upgraded.
Need to get 3427 kB of archives.
After this operation, 17.7 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 poppler-data all 0.4.11-1 [2171 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler118 amd64 22.02.0-2ubuntu0.5 [1071 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 3427 kB in 0s (12.1 MB/s)       
Selecting previously unselected

In [20]:
!pdftohtml -s /kaggle/working/downloaded_pdf.pdf output.html

Page-1
Page-2


In [23]:
!ls -l

# Install Poppler
!apt-get install -y poppler-utils

# Upload PDF file
from google.colab import files
uploaded = files.upload()

# Run pdftohtml to convert the PDF
!pdftohtml -s your_file.pdf output.html

# Verify if the HTML file was created
!ls -l

# Read and inspect the HTML file
from bs4 import BeautifulSoup

with open('output.html', 'r') as file:
    html_content = file.read()

soup = BeautifulSoup(html_content, 'html.parser')
print(soup.prettify())


total 500
-rw-r--r-- 1 root root 259394 Sep 15 17:53 downloaded_pdf.pdf
-rw-r--r-- 1 root root 112836 Sep 15 18:03 output-2_1.png
-rw-r--r-- 1 root root  14792 Sep 15 18:03 output-html.html
-rw-r--r-- 1 root root  13555 Sep 15 18:03 output001.png
-rw-r--r-- 1 root root  50034 Sep 15 18:03 output002.png
-rw-r--r-- 1 root root    633 Sep 15 18:03 outputs.html
-rw-r--r-- 1 root root  44869 Sep 15 17:53 pdfid.py
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
poppler-utils is already the newest version (22.02.0-2ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


ModuleNotFoundError: No module named 'google.colab'

# **Java Script Extract(PdfMinner) 

In [8]:
#!pip install pdfminer.six
!pip install git+https://github.com/pdfminer/pdfminer.six.git

  Cloning https://github.com/pdfminer/pdfminer.six.git to /tmp/pip-req-build-3694kqib
  Running command git clone --filter=blob:none --quiet https://github.com/pdfminer/pdfminer.six.git /tmp/pip-req-build-3694kqib
  fatal: unable to access 'https://github.com/pdfminer/pdfminer.six.git/': Could not resolve host: github.com
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/pdfminer/pdfminer.six.git /tmp/pip-req-build-3694kqib did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/pdfminer/pdfminer.six.git /tmp/pip-req-build-3694kqib did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [7]:
!pip install git+https://github.com/pdfminer/pdfminer.six.git
import requests
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdftypes import resolve1
from io import BytesIO

# Step 1: Download the PDF from the given URL and load into memory
def download_pdf(url):
    response = requests.get(url)
    if response.status_code == 200:
        return BytesIO(response.content)  # Load PDF content in memory as BytesIO object
    else:
        raise Exception(f"Failed to download PDF: {response.status_code}")

# Step 2: Parse PDF and look for embedded JavaScript
def extract_javascript_from_pdf(pdf_data):
    parser = PDFParser(pdf_data)
    document = PDFDocument(parser)

    javascript_code = []

    # Look through the document's catalog for JavaScript
    if 'Names' in document.catalog:
        names = resolve1(document.catalog['Names'])
        if 'JavaScript' in names:
            js_names = resolve1(names['JavaScript'])
            for (key, value) in js_names['Kids']:
                obj = resolve1(value)
                for name, js_obj in obj['Names']:
                    js_code = js_obj.get('JS')
                    if js_code:
                        if isinstance(js_code, bytes):
                            js_code = js_code.decode('utf-8')  # Convert bytes to string
                        javascript_code.append(js_code)

    return javascript_code

# URL of the PDF file
pdf_url = "https://drive.google.com/uc?export=download&id=16ROOOSBt6cUClyMZsKGtVhRlTMN1jKv7"

# Step 3: Download the PDF and load it in memory
pdf_data = download_pdf(pdf_url)

# Step 4: Extract JavaScript from the PDF
javascript_code = extract_javascript_from_pdf(pdf_data)

# Print the extracted JavaScript
if javascript_code:
    for idx, js in enumerate(javascript_code):
        print(f"Extracted JavaScript {idx + 1}:\n{js}\n")
else:
    print("No JavaScript found in the PDF.")


  Cloning https://github.com/pdfminer/pdfminer.six.git to /tmp/pip-req-build-lsblsodx
  Running command git clone --filter=blob:none --quiet https://github.com/pdfminer/pdfminer.six.git /tmp/pip-req-build-lsblsodx
  fatal: unable to access 'https://github.com/pdfminer/pdfminer.six.git/': Could not resolve host: github.com
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/pdfminer/pdfminer.six.git /tmp/pip-req-build-lsblsodx did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/pdfminer/pdfminer.six.git /tmp/pip-req-build-lsblsodx did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


ModuleNotFoundError: No module named 'pdfminer'

# **Java Script Extraxt (Parser) by Fahim**

In [ ]:
# Step 1: Install requests library
!pip install requests

# Step 2: Download pdf-parser.py
!wget https://raw.githubusercontent.com/DidierStevens/DidierStevensSuite/master/pdf-parser/pdf-parser.py -O pdf-parser.py

# Step 3: Verify the download of pdf-parser.py
!ls -l pdf-parser.py  # Check if the file is present

# Import necessary modules
import requests
import subprocess

# Step 4: Function to download PDF from a URL
def download_pdf(url):
    response = requests.get(url)
    if response.status_code == 200:
        with open("downloaded.pdf", "wb") as f:
            f.write(response.content)
        print("PDF downloaded successfully.")
    else:
        print(f"Failed to download PDF: {response.status_code}")

# Step 5: Function to run pdf-parser.py and extract JavaScript
def extract_javascript(pdf_file):
    # Command to run pdf-parser.py to search for "JavaScript"
    command = f"python3 pdf-parser.py --search JavaScript {pdf_file}"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    
    # Print the output of the command
    if result.returncode == 0:
        print("JavaScript extraction output:")
        print(result.stdout)
    else:
        print("Error running pdf-parser.py:")
        print(result.stderr)

# Example Usage
pdf_url = "https://drive.google.com/uc?export=download&id=16ROOOSBt6cUClyMZsKGtVhRlTMN1jKv7"  # Replace with your PDF URL
download_pdf(pdf_url)

# Check if the PDF was downloaded
!ls -l downloaded.pdf  # Verify the downloaded PDF file

extract_javascript("downloaded.pdf")
